In [1]:
import math
import re
import numpy as np
import tensorflow as tf
from collections import Counter

In [2]:
# 如果使用GPU训练，设置memory_growth
gpu=tf.config.experimental.list_physical_devices('GPU')
for gp in gpu:
    tf.config.experimental.set_memory_growth(gp,True)

In [3]:
# 数据路径
DATA_PATH = r'poetry.txt'
# 单行诗最大长度
MAX_LEN = 64
# 禁用的字符，拥有以下符号的诗将被忽略
DISALLOWED_WORDS = ['（', '）', '(', ')', '__', '《', '》', '【', '】', '[', ']']
# 批量大小
BATCH_SIZE = 128

In [4]:
# 一首诗（一行）对应一个列表的元素
poetry = []
# 按行读取数据 poetry.txt
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 遍历处理每一条数据    
for line in lines:
    # 利用正则表达式拆分标题和内容
    fields = re.split(r"[:：]", line)
    # 跳过异常数据
    if len(fields) != 2:
        continue
    # 得到诗词内容（后面不需要标题）
    content = fields[1]
    # 跳过内容过长的诗词
    if len(content) > MAX_LEN - 2:
        continue
    # 跳过存在禁用符的诗词
    if any(word in content for word in DISALLOWED_WORDS):
        continue

    poetry.append(content.replace('\n', '')) # 最后要记得删除换行符

In [6]:
# 最小词频
MIN_WORD_FREQUENCY = 8

# 统计词频，利用Counter可以直接按单个字符进行统计词频
counter = Counter()
for line in poetry:
    counter.update(line)
    
# 过滤掉低词频的词
tokens = [token for token, count in counter.items() if count >= MIN_WORD_FREQUENCY]    

In [7]:
# 补上特殊词标记：填充字符标记、未知词标记、开始标记、结束标记
tokens = ["[PAD]", "[NONE]", "[START]", "[END]"] + tokens
# 映射: 词 -> 编号
word_idx = {}
idx_word = {}
for idx, word in enumerate(tokens):
    word_idx[word] = idx
    idx_word[idx] = word

In [9]:
idx_word[5]

'随'

In [10]:
word_idx['随']

5

In [11]:
class Tokenizer:
    """
    分词器
    """
    def __init__(self, tokens):
        # 词汇表大小
        self.dict_size = len(tokens)
        # 生成映射关系
        self.token_id = {} # 映射: 词 -> 编号
        self.id_token = {} # 映射: 编号 -> 词
        for idx, word in enumerate(tokens):
            self.token_id[word] = idx
            self.id_token[idx] = word

        # 各个特殊标记的编号id，方便其他地方使用
        self.start_id = self.token_id["[START]"]
        self.end_id = self.token_id["[END]"]
        self.none_id = self.token_id["[NONE]"]
        self.pad_id = self.token_id["[PAD]"]

    def id_to_token(self, token_id):
        """
        编号 -> 词
        """
        return self.id_token.get(token_id)

    def token_to_id(self, token):
        """
        词 -> 编号
        """
        return self.token_id.get(token, self.none_id) #编号里没有返回 [NONE]

    def encode(self, tokens):
        """
        词列表 -> [START]编号 + 编号列表 + [END]编号，为甚加[END]，这不是应该自己生成的吗
        """
        token_ids = [self.start_id, ] # 起始标记
        # 遍历，词转编号
        for token in tokens:
            token_ids.append(self.token_to_id(token))
        token_ids.append(self.end_id) # 结束标记
        return token_ids

    def decode(self, token_ids):
        """
        编号列表 -> 词列表(去掉起始、结束标记)
        """
        # 起始、结束标记
        flag_tokens = {"[START]", "[END]"}

        tokens = []
        for idx in token_ids:
            token = self.id_to_token(idx)
            # 跳过起始、结束标记
            if token not in flag_tokens:
                tokens.append(token)
        return tokens

In [12]:
# 完成类
tokenizer = Tokenizer(tokens)

In [13]:
tokenizer.id_to_token(4)

'寒'

In [18]:
'''
构建 DataSet
'''
class PoetryDataSet:
    """
    古诗数据集生成器
    """
    def __init__(self, data, tokenizer, batch_size):
        # 数据集
        self.data = data
        self.total_size = len(self.data)
        # 分词器，用于词转编号
        self.tokenizer = tokenizer
        # 每批数据量
        self.batch_size = batch_size
        # 每个epoch迭代的步数
        self.steps = int(math.floor(len(self.data) / self.batch_size))

    def pad_line(self, line, length, padding=None):
        """
        对齐单行数据
        """
        if padding is None:
            padding = self.tokenizer.pad_id

        padding_length = length - len(line)
        if padding_length > 0:
            return line + [padding] * padding_length
        else:
            return line[:length]

    def __len__(self):
        return self.steps

    def __iter__(self):
        # 打乱数据
        np.random.shuffle(self.data)
        # 迭代一个epoch，每次yield一个batch
        for start in range(0, self.total_size, self.batch_size):
            end = min(start + self.batch_size, self.total_size)
            data = self.data[start:end]
            
            # map根据提供的函数对指定序列做映射
            max_length = max(map(len, data)) 

            batch_data = []
            for str_line in data:
                # 对每一行诗词进行编码、并补齐padding
                encode_line = self.tokenizer.encode(str_line)
                pad_encode_line = self.pad_line(encode_line, max_length + 2) # 加2是因为tokenizer.encode会添加START和END
                batch_data.append(pad_encode_line)

            batch_data = np.array(batch_data)
            # yield 特征、标签
            yield batch_data[:, :-1], batch_data[:, 1:]

    def generator(self):
        while True:
            yield from self.__iter__()

In [19]:
# 初始化 PoetryDataSet
dataset = PoetryDataSet(poetry, tokenizer, BATCH_SIZE)

In [20]:
# 构建模型

model = tf.keras.Sequential([
    # 词嵌入层
    tf.keras.layers.Embedding(input_dim=tokenizer.dict_size, output_dim=150),
    # 第一个LSTM层
    tf.keras.layers.LSTM(150, dropout=0.5, return_sequences=True),
    # 第二个LSTM层
    tf.keras.layers.LSTM(150, dropout=0.5, return_sequences=True),
    # 利用TimeDistributed对每个时间步的输出都做Dense操作(softmax激活)
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(tokenizer.dict_size, activation='softmax')),
])
    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 150)         515100    
_________________________________________________________________
lstm (LSTM)                  (None, None, 150)         180600    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 150)         180600    
_________________________________________________________________
time_distributed (TimeDistri (None, None, 3434)        518534    
Total params: 1,394,834
Trainable params: 1,394,834
Non-trainable params: 0
_________________________________________________________________


In [21]:
def predict(model, token_ids):
    """
    在概率值为前100的词中选取一个词(按概率分布的方式)
    :return: 一个词的编号(不包含[PAD][NONE][START])
    """
    # 预测各个词的概率分布
    # 0  表示对输入的第0个样本做预测
    # -1 表示只要对最新的词的预测
    # 3: 表示不要前面几个标记符
    _probas = model.predict([token_ids, ])[0, -1, 3:]
    # 按概率降序，取前100
    p_args = _probas.argsort()[-100:][::-1] # 此时拿到的是索引
    p = _probas[p_args] # 根据索引找到具体的概率值
    p = p / sum(p) # 归一
    # 按概率抽取一个
    target_index = np.random.choice(len(p), p=p)
    # 前面预测时删除了前几个标记符，因此编号要补上3位，才是实际在tokenizer词典中的编号
    return p_args[target_index] + 3

In [22]:
def generate_random_poem(tokenizer, model, text=""):
    """
    随机生成一首诗
    :param tokenizer: 分词器
    :param model: 古诗模型
    :param text: 古诗的起始字符串，默认为空
    :return: 一首古诗的字符串
    """
    # 将初始字符串转成token_ids，并去掉结束标记[END]
    token_ids = tokenizer.encode(text)[:-1]
    while len(token_ids) < MAX_LEN:
        # 预测词的编号
        target = predict(model, token_ids)
        # 保存结果
        token_ids.append(target)
        # 到达END
        if target == tokenizer.end_id: 
            break

    return "".join(tokenizer.decode(token_ids))

In [23]:
def generate_acrostic_poem(tokenizer, model, heads):
    """
    生成一首藏头诗
    :param tokenizer: 分词器
    :param model: 古诗模型
    :param heads: 藏头诗的头
    :return: 一首古诗的字符串
    """
    # token_ids，只包含[START]编号
    token_ids = [tokenizer.start_id, ]
    # 逗号和句号标记编号
    punctuation_ids = {tokenizer.token_to_id("，"), tokenizer.token_to_id("。")}
    content = []
    # 为每一个head生成一句诗
    for head in heads:
        content.append(head)
        # head转为编号id，放入列表，用于预测
        token_ids.append(tokenizer.token_to_id(head))
        # 开始生成一句诗
        target = -1;
        while target not in punctuation_ids: # 遇到逗号、句号，说明本句结束，开始下一句
            # 预测词的编号
            target = predict(model, token_ids)
            # 因为可能预测到END，所以加个判断
            if target > 3:
                # 保存结果到token_ids中，下一次预测还要用
                token_ids.append(target)
                content.append(tokenizer.id_to_token(target))

    return "".join(content)

In [25]:
class ShowSaveCallback(tf.keras.callbacks.Callback):

    def __init__(self):
        super().__init__()
        # 给一个初始最大值
        self.loss = float("inf")

    def on_epoch_end(self, epoch, logs=None):
        # 保留损失最低的模型
        if logs['loss'] <= self.loss:
            self.loss = logs['loss']
            model.save("./rnn_model.h5")
        # 查看一下本次训练的效果
        print()
        for i in range(5):
            print(generate_random_poem(tokenizer, model))


In [26]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss=tf.keras.losses.sparse_categorical_crossentropy
)

# 开始训练
model.fit(
    dataset.generator(), 
    steps_per_epoch=dataset.steps, 
    epochs=10,
    callbacks=[ShowSaveCallback()]
)

Train for 191 steps
Epoch 1/10
190/191 [============================>.] - ETA: 0s - loss: 5.1105


长水，秋来开，月山。林知高来事君今去白。，。是青，马来。


。清人云出上天风，不不向看出不秋。有。千自酒长秋，出花日，，。同声，生，行独长前别无
青秋。重日一，，。风中白为自，酒无处，。，玉独春秋，，，风
夜生得道。客生，，酒高看有来生夜夜叶事闲春，未君时，，去，见水月。为一，。白深寒下花来在。一
长为，空三，人长，。新生得山，时。
191/191 [==============================] - 45s 237ms/step - loss: 5.1068
Epoch 2/10
190/191 [============================>.] - ETA: 0s - loss: 4.4359
东，，朝。中一明，。。寒花。。。人，高。。，人不春寒犹不。东草。，风空深
处人。，还。西，。。，有里云天，夜何。花。落，声归山寒。，老，月尽见，来
相。玉不。花见草江事，。，事月，风何山，是高事别，城。下，日，落回。。满香来白
此。为无无水草风雨花此，水不三别。，，，行白人无山时路月尽有，应。春。。明明不，，未，闲。
远，时时日人西，酒。风新，，，，知，何。前。，，中。是，雪，更风月来。有。更。此，。为城。。，在。君
191/191 [==============================] - 21s 110ms/step - loss: 4.4354
Epoch 3/10
190/191 [============================>.] - ETA: 0s - loss: 4.2624
一见春云江出情，林前空远去。应寒
三江西，青秋月此子，远金君前多重水，一夜，春人路知未
一心烟如路年，春石。水，上归人月满人，远深。落草前多风。，雨人雨人花，明月叶中时，去见多不，。不从更


山前石风，上云半月山，色不城。马水日。花叶山。深城旧秋。云寒林深。，秋青飞风雨年。下来闲。一山更流。犹更上。此年不水，无相江。
年此年高，雨天飞客。此年雨春入。玉新门出。知家人一。林空新欲。春城城不。
191/191 [==============================] - 24s 127ms/step - loss: 4.2615
Epoch 4/10
190/191 [============================>.] - ETA: 0s - loss: 4.0354
时东秋未草，山复人门天。门草花心处，为云竹云色，东山满多见。野无去下人。江入人上老。
中国多金年城，风与时空山色。玉下在草长草，自月声月为事，夜一来草客尽。
昔自长寒深，清风路在尽。何不无门人，青前生人城。寒为为云外，莫千时见在，坐山去来新。
汉云如青重下。寒山不尽天日。清是君秋城客，高上水鸟上心，
多白秋水草，有天一落事。明生年月远。日山事在落。山事玉成如，只何明客老。
191/191 [==============================] - 19s 97ms/step - loss: 4.0352
Epoch 5/10
190/191 [============================>.] - ETA: 0s - loss: 3.9229
明月草时日，寒道生城路。。道明古上，金君在年，
时月一上飞，秋来不长酒。万上有无晚，幽处月无香。，向前风愁，东来有微。为今暮阳尘。
柳相道未游，江光云天身。草在白青时，，色欲为情。高在时色来。心山得入色，鸟客年年春。
天山花新望，月云未客流。相林独知鸟，花向有落，日酒春稀。闲夜雪月色。花清花出不，此此春有空。
大门水何风，西风白寒游。无子不寒，明已来，无水树新。心似生家，天相路月。身闲过愁时，别山年心亲。
191/191 [==============================] - 18s 94ms/step - loss: 3.9231
Epoch 6/10
190/191 [============================>.] - ETA: 0s - loss: 3.8245
时水月花门，青时水春斜。自中春人来，花长寒风头。长子一花过，玉应玉多前。
朝马相门远远路，欲无来客犹空门。唯君事应无

In [43]:
# 加载模型
model = tf.keras.models.load_model("rnn_model.h5")

In [49]:
generate_random_poem(tokenizer, model, text='床前明月光')

'床前明月光，野树隔庭扉。旧有谁是雁，高人事过吟。多生仙山落，明风望还鱼。自遣无时尽，何家更话来。'

In [53]:
generate_acrostic_poem(tokenizer, model, heads='深度学习')

'深地白霜晚，度日半长舟。学酒随城里，习事无子烟。'